In [4]:
import numpy as np
import sklearn.metrics
import keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Para descomprimir el archivo Neumonia.zip (Si se usa desde Colab)
# Creará dos carpetas:
#     "test"
#     "train"

!unzip "Neumonia.zip" -d "."

In [ ]:
# Generadores
# El parámetro rescale se encarga de escalar los valores del rango [0..255] a [0..1]
train_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_folderpath = "train/"
test_folderpath = "test/"

# Tamaño de las imágenes a procesar (resized)
alto, ancho = 32, 32
canales = 3 # número de canales

batch_size = 24

print("Train data")
train_generator = train_datagen.flow_from_directory(
    train_folderpath, # directorio de donde cargar las imagenes (train)
    target_size=(alto, ancho),
    batch_size=batch_size,
    class_mode='binary'
    )

n_train = train_generator.samples

print("\n\nTest data")
test_generator = test_datagen.flow_from_directory(
    test_folderpath, # directorio de donde cargar las imagenes (val)
    target_size=(alto, ancho),
    batch_size=batch_size,
    class_mode='binary')

n_test = test_generator.samples

print("\n\nCodificación de clases")
class_dict = train_generator.class_indices # un diccionario donde "key" es el nombre de la clase (nombre del directorio) y "value" es el valor de la clase (0,1, etc.)
print(class_dict)

class_dict_inverse = {}  # Construyo un diccionario donde "key" es el valor de la clase (0, 1, etc.) y "value" es el nombre de la clase (nombre del directorio)
for key in class_dict:
    value = class_dict[key]
    class_dict_inverse[value] = key

In [ ]:
d_in = (alto, ancho, canales)
d_out = 1 # Clasificación binaria

modelo = keras.Sequential([
    keras.layers.Conv2D(8, input_shape=d_in, kernel_size=3, padding='same', activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation= 'tanh'),
    keras.layers.Dense(d_out, activation="sigmoid")
	])

modelo.compile(
  optimizer = keras.optimizers.SGD(learning_rate=0.01),
  loss = 'binary_crossentropy', metrics = ['accuracy']
)

modelo.summary()

In [ ]:
history = modelo.fit(train_generator, steps_per_epoch = n_train // batch_size,
                    epochs=5,  validation_data = test_generator  )

In [ ]:
test_generator.reset()
results = modelo.predict(test_generator)

In [ ]:
test_file_names = test_generator.filenames
test_labels = test_generator.labels # clase de cada muestra de test

preds = np.zeros(results.shape)
aciertos = 0
umbral = 0.5
for index, pred in enumerate(results):
    if(pred < umbral):
        preds[index] = 0
    else:
        if(pred > (1 - umbral)):
            preds[index] = 1
    if (preds[index] == test_labels[index]):
      aciertos = aciertos + 1
print(f"{aciertos} aciertos en {test_labels.shape[0]} muestras de test.")
print(f"Accuracy: {float(aciertos)/test_labels.shape[0]}")

In [ ]:
print('TRUE CLASS\tPREDICTED CLASS\tFILENAME')
for i, p in enumerate(preds):
    pred_index = p[0]
    pred_class = class_dict_inverse[pred_index]
    true_class = class_dict_inverse[test_labels[i]]
    file = test_file_names[i]
    print(f'{true_class}\t{pred_class}\t{file}')

In [ ]:
sklearn.metrics.ConfusionMatrixDisplay.from_predictions(test_labels, preds)